In [1]:
!pip install mlforecast xgboost scikit-optimize


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip freeze > ..//requirements.txt

# regressores

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', decimal=',', thousands='.', parse_dates=True)[2][1:]
df.columns=['Date','Preco']
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df_petroleo = df.sort_values('Date')

# ajustar df de petroleo
df_petroleo['Year'] = [str(x.year) for x in df_petroleo['Date']]
df_petroleo['Preco'] = df_petroleo['Preco'].astype(float) 
df_petroleo_year = df_petroleo.groupby('Year').agg({'Preco':'mean'}).reset_index()
df_petroleo_year['Country Name'] = 'Price'

# conflitos
- temos apenas 3 anos, nao vamos usar

# fuel cons

In [5]:
#  selecionar colunas numericas e nome
cols = [str(x) for x in ['Country Name','Region'] + list(range(1960, 2023))]
df_fuel_cons = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fossil_fuel_consumption/API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_6299038.csv')
df_country_region = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fossil_fuel_consumption/Metadata_Country_API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_6299038.csv')[['Country Code','Region']].dropna()
df_fuel_cons = df_fuel_cons.merge(df_country_region, how='inner', on='Country Code').drop(columns={'Country Code'})[cols]

# dropar na
df_fuel_cons = df_fuel_cons.dropna(axis=1, thresh=0.95)

df_fuel_cons = df_fuel_cons\
    .melt(id_vars=['Country Name','Region'])\
    .rename(columns={'variable':'Year', 'value':'Fuel Consumption'})\
    .drop(columns='Region')\
    .pivot(index='Year', columns='Country Name')\
    .reset_index()
df_fuel_cons.columns = ['_'.join(col) for col in df_fuel_cons.columns.values]
df_fuel_cons = df_fuel_cons.rename(columns={'Year_':'Year'})


# energy use

In [6]:
df_uso_energia_or = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/energy_use/API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_6301176.csv').drop(columns={'Indicator Name','Indicator Code'})
df_country_region = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/energy_use/Metadata_Country_API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_6301176.csv')[['Country Code','Region']].dropna()
df_uso_energia_or = df_uso_energia_or.merge(df_country_region, how='inner', on='Country Code').drop(columns={'Country Code','Region'})
df_uso_energia_or = df_uso_energia_or.dropna(axis=1, thresh=0.95)
df_uso_energia_or = df_uso_energia_or\
    .melt(id_vars=['Country Name'])\
    .rename(columns={'variable':'Year', 'value':'Energy Use'})\
    .pivot(index='Year', columns='Country Name')\
    .reset_index()
df_uso_energia_or.columns = ['_'.join(col) for col in df_uso_energia_or.columns.values]
df_uso_energia_or = df_uso_energia_or.rename(columns={'Year_':'Year'})

# exportacao

# fuel exports

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime


cols = [str(x) for x in ['Country Name'] + list(range(1960, 2023))]


In [8]:
#  selecionar colunas numericas e nome
cols = [str(x) for x in ['Country Name','Region'] + list(range(1960, 2023))]
df_fuel_exp = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fuel_exports/API_TX.VAL.FUEL.ZS.UN_DS2_en_csv_v2_6302702.csv')
df_country_region = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fuel_exports/Metadata_Country_API_TX.VAL.FUEL.ZS.UN_DS2_en_csv_v2_6302702.csv')[['Country Code', 'Region']].dropna()
df_fuel_exp = df_fuel_exp.merge(df_country_region, how='inner', on='Country Code').drop(columns={'Country Code'})[cols]

# dropar na
df_fuel_exp = df_fuel_exp.dropna(axis=1, thresh=0.95).drop(columns={'Region'})
df_fuel_exp = df_fuel_exp\
    .melt(id_vars=['Country Name'])\
    .rename(columns={'variable':'Year', 'value':'Fuel Exports'})\
    .pivot(index='Year', columns='Country Name')\
    .reset_index()
df_fuel_exp.columns = ['_'.join(col) for col in df_fuel_exp.columns.values]
df_fuel_exp = df_fuel_exp.rename(columns={'Year_':'Year'})
df_fuel_exp

,Year,Fuel Exports_Afghanistan,Fuel Exports_Albania,Fuel Exports_Algeria,Fuel Exports_American Samoa,Fuel Exports_Andorra,Fuel Exports_Angola,Fuel Exports_Antigua and Barbuda,Fuel Exports_Argentina,Fuel Exports_Armenia,...,Fuel Exports_Uruguay,Fuel Exports_Uzbekistan,Fuel Exports_Vanuatu,"Fuel Exports_Venezuela, RB",Fuel Exports_Viet Nam,Fuel Exports_Virgin Islands (U.S.),Fuel Exports_West Bank and Gaza,"Fuel Exports_Yemen, Rep.",Fuel Exports_Zambia,Fuel Exports_Zimbabwe
0,1962,NaN,NaN,NaN,NaN,NaN,3.040736,NaN,1.080847,NaN,...,NaN,NaN,NaN,92.362928,NaN,NaN,NaN,NaN,NaN,NaN
1,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.822615,NaN,...,NaN,NaN,NaN,92.911190,NaN,NaN,NaN,NaN,NaN,NaN
2,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.432269,NaN,...,NaN,NaN,NaN,91.099545,NaN,NaN,NaN,NaN,NaN,NaN
3,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615569,NaN,...,NaN,NaN,NaN,92.518557,NaN,NaN,NaN,NaN,NaN,NaN
4,1966,NaN,NaN,59.292765,NaN,NaN,NaN,NaN,0.878622,NaN,...,NaN,NaN,NaN,93.401158,NaN,NaN,NaN,NaN,0.016166,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2018,10.213181,1.662666,NaN,NaN,0.013638,95.672320,0.000000,4.119374,3.730996,...,1.194233,24.939491,NaN,NaN,1.613913,NaN,0.071807,NaN,1.015304,1.010572
57,2019,8.051279,0.602741,NaN,NaN,0.090627,95.249248,0.000000,3.597788,2.850174,...,1.085644,16.559885,NaN,NaN,1.377051,NaN,0.086292,0.145219,1.367831,1.126097
58,2020,NaN,0.905736,NaN,NaN,0.029097,93.051290,16.831744,2.678988,3.002052,...,1.240841,5.958746,NaN,NaN,0.948499,NaN,0.107838,NaN,1.738531,1.102937
59,2021,NaN,0.023915,NaN,NaN,0.033062,94.163746,0.000000,2.642793,1.844380,...,5.650352,6.531394,NaN,NaN,0.979645,NaN,0.068566,NaN,1.460417,1.563256


# dow jones e nasdaq

In [9]:
# dado de dow jones e nasdaq
df_dowjones_nasdaq = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/df_brent_dowjones_nasdaq.csv')
df_dowjones_nasdaq['Year'] = [str(pd.to_datetime(x).year) for x in df_dowjones_nasdaq['DATE']]
df_dowjones_nasdaq = df_dowjones_nasdaq\
    .groupby('Year')\
    .agg({
          'value_dow_jones':('mean', 'median', 'std','min','max')
        , 'value_nasdaq':('mean', 'median', 'std','min','max')
    })\
    .reset_index()
df_dowjones_nasdaq.columns = ['_'.join(col) for col in df_dowjones_nasdaq.columns.values]
df_dowjones_nasdaq = df_dowjones_nasdaq.rename(columns={'Year_':'Year'})
df_dowjones_nasdaq

,Year,value_dow_jones_mean,value_dow_jones_median,value_dow_jones_std,value_dow_jones_min,value_dow_jones_max,value_nasdaq_mean,value_nasdaq_median,value_nasdaq_std,value_nasdaq_min,value_nasdaq_max
0,1987,2308.118009,2420.850000,293.450817,1738.740000,2722.420000,396.090664,425.880,56.130744,291.88,455.26
1,1988,2059.767814,2067.390000,71.951844,1879.140000,2183.500000,374.181475,376.760,14.759840,330.47,396.11
2,1989,2510.009836,2514.610000,180.865397,2144.640000,2791.410000,437.739370,446.470,26.910854,378.56,485.73
3,1990,2679.243178,2659.130000,146.901064,2365.100000,2999.750000,409.248630,425.440,43.043651,325.44,469.60
4,1991,2928.487726,2947.230000,122.798514,2470.300000,3168.830000,491.574055,497.000,45.711536,355.75,586.34
5,1992,3283.772033,3280.410500,60.358959,3136.583000,3414.678000,599.288005,588.475,31.539596,547.84,676.95
6,1993,3522.636479,3527.427394,131.664351,3241.946535,3794.328275,714.835781,702.710,37.172420,645.87,787.42
7,1994,3792.561261,3797.427365,87.781492,3593.353775,3978.357635,751.593781,749.530,27.609827,693.79,803.93
8,1995,4494.531388,4585.143620,384.176415,3832.080950,5216.468613,925.206521,938.870,109.272103,743.58,1069.79
9,1996,5742.304054,5662.778648,352.771612,5032.940249,6560.902590,1165.955546,1162.080,80.574543,988.57,1316.27


# tudo junto

In [10]:

from sklearn.preprocessing import MinMaxScaler

In [11]:
df_final = df_fuel_cons\
    .merge(df_uso_energia_or, on='Year', how='inner')\
    .merge(df_petroleo_year[['Year','Preco']], on='Year', how='inner')\
    .merge(df_fuel_exp, on='Year', how='inner')\
    .merge(df_dowjones_nasdaq, on='Year', how='inner')\
    .replace(0.0, np.nan)\
    .dropna(axis=0, thresh=0.5)\
    .dropna(axis=1, thresh=0.5)
df_final

,Year,Fuel Consumption_Albania,Fuel Consumption_Algeria,Fuel Consumption_Angola,Fuel Consumption_Argentina,Fuel Consumption_Armenia,Fuel Consumption_Australia,Fuel Consumption_Austria,Fuel Consumption_Azerbaijan,Fuel Consumption_Bahrain,...,value_dow_jones_mean,value_dow_jones_median,value_dow_jones_std,value_dow_jones_min,value_dow_jones_max,value_nasdaq_mean,value_nasdaq_median,value_nasdaq_std,value_nasdaq_min,value_nasdaq_max
0,1987,78.637511,99.755816,20.384236,88.161659,NaN,93.738609,79.578858,NaN,NaN,...,2308.118009,2420.850000,293.450817,1738.740000,2722.420000,396.090664,425.880,56.130744,291.88,455.26
1,1988,80.381404,99.898313,22.017965,90.181006,NaN,93.578428,74.693154,NaN,100.000000,...,2059.767814,2067.390000,71.951844,1879.140000,2183.500000,374.181475,376.760,14.759840,330.47,396.11
2,1989,79.693812,99.862972,22.103251,91.379251,NaN,93.852171,76.544349,NaN,99.999980,...,2510.009836,2514.610000,180.865397,2144.640000,2791.410000,437.739370,446.470,26.910854,378.56,485.73
3,1990,76.593975,99.938296,25.480388,88.652605,97.044055,93.911286,79.132345,99.904994,99.822091,...,2679.243178,2659.130000,146.901064,2365.100000,2999.750000,409.248630,425.440,43.043651,325.44,469.60
4,1991,69.824996,NaN,25.482861,88.854739,96.516567,93.688718,79.127007,99.902953,NaN,...,2928.487726,2947.230000,122.798514,2470.300000,3168.830000,491.574055,497.000,45.711536,355.75,586.34
5,1992,56.079291,NaN,24.098076,88.805945,92.719285,94.342660,77.237211,99.321011,99.917853,...,3283.772033,3280.410500,60.358959,3136.583000,3414.678000,599.288005,588.475,31.539596,547.84,676.95
6,1993,53.478978,NaN,24.964662,87.687229,82.063767,93.758346,77.011142,98.585026,NaN,...,3522.636479,3527.427394,131.664351,3241.946535,3794.328275,714.835781,702.710,37.172420,645.87,787.42
7,1994,54.846915,NaN,24.439683,87.001935,76.968403,93.886483,77.585708,98.664796,99.970649,...,3792.561261,3797.427365,87.781492,3593.353775,3978.357635,751.593781,749.530,27.609827,693.79,803.93
8,1995,49.541641,99.872459,22.780196,87.092096,84.251064,93.831770,77.982799,98.659733,99.999984,...,4494.531388,4585.143620,384.176415,3832.080950,5216.468613,925.206521,938.870,109.272103,743.58,1069.79
9,1996,45.453764,99.841535,23.498097,87.696284,57.650824,93.648221,78.198187,98.394267,99.942058,...,5742.304054,5662.778648,352.771612,5032.940249,6560.902590,1165.955546,1162.080,80.574543,988.57,1316.27


In [12]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from skopt import BayesSearchCV

X, y = df_final.drop(columns=['Preco']), df_final['Preco']

numeric_features = list(set(X.columns) - set(['Year']))
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

categorical_features = ["Year"]
categorical_transformer = Pipeline(
    steps=[
          ("encoder", OneHotEncoder(handle_unknown="ignore"))
        # , ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

predict_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", XGBRegressor(seed=42))]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

predict_pipeline.fit(X_train, y_train)
print(f'score={predict_pipeline.score(X_test, y_test)}')
print(f'mape={mean_absolute_percentage_error(y_test, predict_pipeline.predict(X_test))}')
predict_pipeline

score=0.8864485823784491
mape=0.2186251076369997


d:\Cursos\FIAP_pós\gp27_techchallenge_4\.venv\lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['Fuel Exports_Cabo Verde']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
d:\Cursos\FIAP_pós\gp27_techchallenge_4\.venv\lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['Fuel Exports_Cabo Verde']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
d:\Cursos\FIAP_pós\gp27_techchallenge_4\.venv\lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['Fuel Exports_Cabo Verde']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Fuel Exports_Israel',
                                                   'Fuel Exports_Georgia',
                                                   'Fuel Exports_India',
                                                   'Fuel Consumption_Eritrea',
                                                   'Fuel Exports_Australia',
                                                   'Fuel Exports_Andorra',
                                                   'Fuel Exports_Guinea',
                                                   'Fuel Consumption_Dominican '
                                                   'Republic',
                                                   'Fuel Consump...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [13]:
import joblib
joblib.dump(predict_pipeline, 'D:/Cursos/FIAP_pós/gp27_techchallenge_4/models/xgb_model.pkl')

['D:/Cursos/FIAP_pós/gp27_techchallenge_4/models/xgb_model.pkl']

In [14]:
from sklearn.utils import estimator_html_repr
with open("D:/Cursos/FIAP_pós/gp27_techchallenge_4/models/pipeline.html", "w", encoding='utf-8') as f:
    f.write(estimator_html_repr(predict_pipeline))

In [15]:
model = predict_pipeline['regressor']
df_importances = pd.DataFrame([predict_pipeline[:-1].get_feature_names_out(), model.feature_importances_], index=['Features','Importance']).T

In [16]:
df_importances.loc[df_importances.Importance > 0.000001].sort_values('Importance', ascending=False)

,Features,Importance
2,num__Fuel Exports_India,0.462135
8,num__Fuel Consumption_Bolivia,0.234046
17,num__value_dow_jones_mean,0.199892
86,num__Fuel Exports_Poland,0.033211
223,num__Fuel Consumption_Sudan,0.032618
1,num__Fuel Exports_Georgia,0.012022
81,num__Fuel Exports_Germany,0.009429
0,num__Fuel Exports_Israel,0.008297
122,num__Fuel Exports_Denmark,0.006668
36,num__Fuel Exports_Jordan,0.000762
